In [1]:
##基于用户的协同过滤
import math
import numpy as np 
rec_size =50 #推荐集数

In [15]:
##读取uid list
def load_uid():
    uid_list = []
    with open('../../data/uid.txt','r',encoding = 'utf-8') as f1:
        while True:
            line = f1.readline()
            if not line:
                break
            uid_list.append(line.strip())
        f1.close()
    return uid_list

In [16]:
##最大最小归一化
def max_min(data):
    ma = 0
    mi = 100
    for item in data.keys():
        if data[item] > ma:
            ma = data[item]
        if data[item] < mi:
            mi = data[item]
    for item in data.keys():
        data[item] = (data[item] - mi) / (ma-mi)
    return data

In [17]:
##读取目标用户关注列表
def load_follow():
    follow = []
    filename = '../../data/sample/user_'+str(uid)+'_follower.dat'
    with open(filename,'r',encoding = 'utf-8') as f:
        follow = f.readline().strip()
        follow = eval(follow)
        f.close()
    return follow

In [18]:
##读取目标用户互相关注列表
def load_huguan():
    filename = '../../data/sample/user_'+str(uid)+'_huguan.dat'
    with open(filename,'r',encoding = 'utf-8') as f:
        huguan = f.readline().strip()
        huguan = eval(huguan)
        f.close()
    return huguan

In [19]:
##读入训练集
def load_train():
    uid_wid = {}
    original = {}
    author = {}
    text = {}
    filename = '../../data/train/'+str(uid)+'.dat'
    with open(filename,'r',encoding='utf-8') as f:
        while True:
            line = f.readline()
            if not line:
                break
            line = eval(line)
            wid = line['id']
            userid = line['userid']
            originalid = line['original']
            if 'id' not in originalid:
                original[wid] = wid
                author[wid] = userid
                text[wid] = line['text']
            else:
                original[wid] = originalid['id']
                text[original[wid]] = originalid['text']
            if userid not in uid_wid:
                uid_wid[userid] = []
            uid_wid[userid].append(original[wid])
        f.close()
    return uid_wid,original,author,text

In [20]:
##读入测试集
def load_test():
    test = []
    user_test = []
    filename = '../../data/test/'+str(uid)+'.dat'
    with open(filename,'r',encoding='utf-8') as f:
        while True:
            line = f.readline()
            if not line:
                break
            line = eval(line)
            wid = line['id']
            userid = line['userid']
            originalid = line['original']
            if 'id' not in originalid:
                original[wid] = wid
                author[wid] = userid
                
            else:
                original[wid] = originalid['id']
            if userid not in uid_wid:
                uid_wid[userid] = []
            uid_wid[userid].append(original[wid])
            test.append(original[wid])
            if userid == uid:
                user_test.append(original[wid])
            if original[wid] not in wid_uid:
                wid_uid[original[wid]] = []
            wid_uid[original[wid]].append(userid)
        f.close()
    return test,user_test

In [21]:
##微博-用户矩阵
def get_wid_uid():
    wid_uid = {}
    for u in uid_wid.keys():
        for w in uid_wid[u]:
            if w not in wid_uid:
                wid_uid[w] = []
            wid_uid[w].append(u)
    return wid_uid

In [22]:
##交互相似度
def interact_sim():
    sim = {}
    for u in follow:
        sim[int(u)] = 0
    for w in uid_wid[uid]:##目标用户的每条微博
        for u in wid_uid[w]: ##产生行为的用户
            if u == uid:
                continue
            sim[u] += 1
        if w in author and author[w] != uid: ##原创作者相似度再加一
            sim[author[w]] += 1
    for u in sim:
        if u not in uid_wid:
            continue
        sim[u] /= math.sqrt(len(uid_wid[u]))
    return sim

In [23]:
##保存召回结果
def save_result(result):
    filename = './weibo_by_CF/'+str(uid)+'.dat'
    with open(filename,'w',encoding='utf-8') as f:
        for item in result:
            f.write(str(item)+'\n')
        f.close()

In [24]:
def get_result(sim):
    score = {}
    for wid in test:
        score[wid] = 0
        for u in wid_uid[wid]:
            if u == uid:
                continue         
            score[wid] += sim[u]
    score = sorted(score.items(),key=lambda x:x[1],reverse=True)
    result = set()
    weibo = []
    for i in range(0,rec_size):
        if score[i][1] <= 0:
            break
        result.add(score[i][0])
    count = 0
    for item in result:
        if item in user_test:
            count += 1
    save_result(result)
    return count

In [25]:
##保存用户相似度
def save_sim():
    filename = './user_sim/'+str(uid)+'.dat'
    with open(filename,'w',encoding='utf-8') as f:
        for item in sim.keys():
            f.write(str(item)+'\t'+str(sim[item])+'\n')
        f.close()

In [26]:
if __name__ == '__main__':
    uid_list = load_uid()
    for rec_size in [10,20,30,40,50]:
        content_count = 0
        fam_count = 0
        interact_count = 0
        res = {}
        count = 0
        for uid in uid_list:
            uid = int(uid)
            uid_wid,original,author,text = load_train()  ##读取训练集
            huguan = load_huguan() ##读取互关列表
            wid_uid = get_wid_uid() ##微博-用户矩阵
            follow = load_follow() ##获取关注用户列表
            sim = interact_sim() ##相似度
            save_sim()
            test,user_test = load_test()  ##获取测试集
            count += get_result(sim)
        print("top{}, recall: {}".format(rec_size,count/800)) 
        print("top{}, precision: {}".format(rec_size,count/(rec_size*20)))

top10, recall: 0.085
top10, precision: 0.34
top20, recall: 0.145
top20, precision: 0.29
top30, recall: 0.205
top30, precision: 0.2733333333333333
top40, recall: 0.25
top40, precision: 0.25
top50, recall: 0.275
top50, precision: 0.22
